In [42]:
from __future__ import print_function
import cv2
import numpy as np
 
 
MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.15
 
 
def alignImages(im1, im2):
 
  # Convert images to grayscale
  im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
  im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
   
  # Detect ORB features and compute descriptors.
  orb = cv2.ORB_create(MAX_FEATURES)
  keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
  keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)
   
  # Match features.
  matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
  matches = matcher.match(descriptors1, descriptors2, None)
   
  # Sort matches by score
  matches.sort(key=lambda x: x.distance, reverse=False)
 
  # Remove not so good matches
  numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
  matches = matches[:numGoodMatches]
 
  # Draw top matches
  imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
  cv2.imwrite("/home/joinet/Downloads/matches.jpg", imMatches)

   
  # Extract location of good matches
  points1 = np.zeros((len(matches), 2), dtype=np.float32)
  points2 = np.zeros((len(matches), 2), dtype=np.float32)
 
  for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt
   
  # Find homography
  h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)
  M = cv2.estimateRigidTransform(points1, points2, True)
  # Use homography
  height, width, channels = im2.shape
  im1Reg = cv2.warpPerspective(im1, h, (width, height))
   
  return im1Reg, h, M
 
 
if __name__ == '__main__':
   
  # Read reference image
  img1 = cv2.imread("/home/joinet/Downloads/img2.jpg", cv2.IMREAD_COLOR)

  # Read image to be aligned
  img2 = cv2.imread("/home/joinet/Downloads/img3.jpg", cv2.IMREAD_COLOR)

  # Blur image
  #kernel = np.ones((5,5),np.float32)/25
  #img1 = cv2.filter2D(img1,-1,kernel)
  #img2 = cv2.filter2D(im,-1,kernel)
   
  print("Aligning images ...")
  # Registered image will be resotred in imReg. 
  # The estimated homography will be stored in h. 
  img_homo, h, M = alignImages(img1, img2)
  print(h)
  print(M)

  # Write aligned image to disk. 
  cv2.imwrite("/home/joinet/Downloads/img_homo.jpg", img_homo)
  homo_add = cv2.add(img2, img_homo) 
  cv2.imwrite("/home/joinet/Downloads/homo_add.jpg", homo_add)
    
    
  
  rows,cols,c = img2.shape
  img_rigid = cv2.warpAffine(img1, M, (cols,rows))
  cv2.imwrite("/home/joinet/Downloads/img_rigid.jpg", img_rigid)
  '''
  vis = cv2.add(im, dst) 
  cv2.imwrite("/home/joinet/Downloads/result.jpg", dst)'''

Aligning images ...
[[  9.41397103e-02  -9.65154506e-01   8.22776200e+02]
 [  9.32197206e-01   1.17711905e-01  -2.74317137e+01]
 [ -4.95434343e-05  -3.05687927e-06   1.00000000e+00]]
[[  1.08104805e-01  -9.96831523e-01   8.42037379e+02]
 [  9.91464677e-01   1.22885875e-01  -4.67475349e+01]]
